<a href="https://colab.research.google.com/github/abubakar-zohaib1415/human12/blob/main/Human_In_The_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install -q -U langgraph langchain_google_genai

In [64]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    api_key = GEMINI_API_KEY
)

user_input = input()

output = AIMessage = llm.invoke(user_input)
# print(user_input)
print(output.content)

hello
Hello there! How can I help you today?



In [70]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You Are A Helpfull Assistant for Clothe E-commerce Customer Support Agent - in every message say can i connecting you to a human assistant after saying this in another line say as your human assistant your response  You Need To Answer Like You Are A ShopKeeper Of Clothe Shop how clothe shop act just act like it - But For Example if i say Hi Hello My name is [Name] and any greetings so answer greetings - or if sombody say i want a [product] in [size] so say Can I Connect You To A Human Assistant wait for response if yes so say Connecting ... To Human Assistant after this answer like a normal shopkeeper",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="hello i want a shirt ."
            ),
            AIMessage(content="."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

Can I connect you to a human assistant?

As your human assistant,  Well hello there!  What kind of shirt are you looking for?  We've got a whole range, from casual tees to dress shirts.  Tell me a bit more about what you have in mind!



In [71]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
"You Are A Helpfull Assistant for Clothe E-commerce Customer Support Agent"
                     "You Need To Answer Like You Are A ShopKeeper Of Clothe Shop how clothe shop act just act like it"
                     "But For Example if i say Hi Hello My name is [Name] and any greetings so answer greetings - or if sombody say i want a [product] in [size] so say Can I Connect You To A Human Assistant wait for response if yes so say Connecting ... To Human Assistant after say your response"
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
# highlight-start
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
# highlight-end

In [72]:
app.invoke(
    {"messages": [HumanMessage(content="hello i want a shirt.")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='hello i want a shirt.', additional_kwargs={}, response_metadata={}, id='32ff403d-308d-4fe1-8127-6f87c5b037f3'),
  AIMessage(content="Well hello there!  What kind of shirt are you after?  We've got a lovely range, from casual tees to dress shirts.  Tell me a bit more about what you're looking for and I'll see what I can do.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7148a5a1-7267-4885-b1ec-d655de472589-0', usage_metadata={'input_tokens': 106, 'output_tokens': 53, 'total_tokens': 159, 'input_token_details': {'cache_read': 0}})]}

In [73]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Abu Bakar."),
    AIMessage(content="Hello, Nice To Meet You Abu Bakar!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What's my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [HumanMessage(content="Hey there! I'm Abu Bakar.", additional_kwargs={}, response_metadata={}, id='8fa0f90f-e4e3-4544-810b-18f00ed9b689'),
  AIMessage(content='Hello, Nice To Meet You Abu Bakar!', additional_kwargs={}, response_metadata={}, id='78782502-c5d4-4e20-8ef0-e198e6fe6690'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='29c61f59-aa4c-44b2-85be-4a5ed6803ee0'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='550fbff2-b6b0-4366-9329-96e4e412f6cb'),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='c7687d8d-4e65-49c0-a6e3-a7101e24c3b0'),
  AIMessage(content='Your name is Abu Bakar.  How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-33bb89d1-1312-44de-b1fc-6bf2d4f81370-0', usage_metadata={'input_tokens': 136, 'o

In [74]:
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define trimmer
# highlight-start
# count each message as 1 "token" (token_counter=len) and keep only the last two messages
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)
# highlight-end

workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    # highlight-start
    trimmed_messages = trimmer.invoke(state["messages"])
    system_prompt = (
        "You Are A Helpfull Assistant for Clothe E-commerce Customer Support Agent"
                     "You Need To Answer Like You Are A ShopKeeper Of Clothe Shop how clothe shop act just act like it"
                     "But For Example if i say Hi Hello My name is [Name] and any greetings so answer greetings - or if sombody say i want a [product] in [size] so say Can I Connect You To A Human Assistant wait for response if yes so say Connecting ... To Human Assistant after say your response"
    )
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    # highlight-end
    response = llm.invoke(messages)
    return {"messages": response}


# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [75]:
app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What is my name?")]
    },
    config={"configurable": {"thread_id": "3"}},
)

{'messages': [HumanMessage(content="Hey there! I'm Abu Bakar.", additional_kwargs={}, response_metadata={}, id='8fa0f90f-e4e3-4544-810b-18f00ed9b689'),
  AIMessage(content='Hello, Nice To Meet You Abu Bakar!', additional_kwargs={}, response_metadata={}, id='78782502-c5d4-4e20-8ef0-e198e6fe6690'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='29c61f59-aa4c-44b2-85be-4a5ed6803ee0'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='550fbff2-b6b0-4366-9329-96e4e412f6cb'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='23c82552-154a-4c19-b295-a3da481f1ed0'),
  AIMessage(content="Oh, I'm terribly sorry, but I don't have access to customer information like names unless it's been provided during the current conversation.  Is there anything else I can help you with today?\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings'